<img src='./img/EU-Copernicus-EUM_3Logos.png' alt='Logo EU Copernicus EUMETSAT' align='right' width='40%'></img>

<br>

# Functions

This notebook lists all `functions` that are defined and used during the `Fire Monitoring course`.
The following functions are listed:

**[Data loading and re-shaping functions](#load_reshape)**
* [generate_masked_array](#generate_masked_array)
* [slstr_frp_gridding](#slstr_frp_gridding)

**[Data visualization functions](#visualization)**
* [visualize_pcolormesh](#visualize_pcolormesh)
* [visualize_s3_frp](#vis_s3_frp)

<hr>

#### Load required libraries

In [1]:
import os
from matplotlib import pyplot as plt

import xarray as xr
from netCDF4 import Dataset
import numpy as np
import glob

from matplotlib import pyplot as plt
import matplotlib.colors
import matplotlib.cm as cm
from matplotlib.colors import LogNorm
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature

import warnings
warnings.simplefilter(action = "ignore", category = RuntimeWarning)
warnings.simplefilter(action = "ignore", category = FutureWarning)

<hr>

## <a id="load_reshape"></a>Data loading and re-shaping functions

### <a id='generate_masked_array'></a>`generate_masked_array`

In [10]:
def generate_masked_array(xarray, mask, threshold, operator, drop=True):
    """ 
    Applies a mask (e.g. cloud fraction values or masking out certain data values) onto a given data array, based on a given threshold.
    
    Parameters:
        xarray (xarray DataArray): a three-dimensional xarray DataArray object
        mask (xarray DataArray): 1-dimensional xarray DataArray, e.g. cloud fraction values
        threshold (float): any number between 0 and 1, specifying the degree of cloudiness which is acceptable
        operator (str): operator how to mask the array, e.g. '<', '>' or '='
        drop(boolean): whether to drop the values that are masked out. Default is True.
        
    Returns:
        Masked xarray DataArray with flagged negative values
    """
    if(operator=='<'):
        cloud_mask = xr.where(mask < threshold, 1, 0) #Generate cloud mask with value 1 for the pixels we want to keep
    elif(operator=='!='):
        cloud_mask = xr.where(mask != threshold, 1, 0)
    elif(operator=='>'):
        cloud_mask = xr.where(mask > threshold, 1, 0)
    else:
        cloud_mask = xr.where(mask == threshold, 1, 0)
            
    xarray_masked = xr.where(cloud_mask ==1, xarray, np.nan) #Apply mask onto the DataArray
    print(xarray_masked)
    xarray_masked.attrs = xarray.attrs #Set DataArray attributes 
    if(drop):
        return xarray_masked[~np.isnan(xarray_masked)] #Return masked DataArray and flag negative values
    else:
        return xarray_masked

### <a id='slstr_frp_gridding'></a>`slstr_frp_gridding`

In [1]:
def slstr_frp_gridding(parameter_array, lat_frp, lon_frp, parameter, lat_min, lat_max, lon_min, lon_max, sampling_lat, sampling_lon, n_fire, **kwargs):
    """ 
    Grids the NRT Sentinel-3 SLSTR Fire Radiative Power data onto a regular latitude and longitude data
    
    Parameters:
        parameter_array (xarray DataArray): xarray DataArray object
        lat_frp (xarray DataArray): latitude information retrieved from the data file
        lon_frp (xarray DataArray): longitude information retrieved from the data file
        parameter (str): Parameter of the SLSTR FRP data
        lat_min (float): latitude minimum
        lat_max (float): latitude maximum
        lon_min (float): longitude minimum
        lon_max (float): longitude maximum
        sampling_lat(float): resolution for latitude
        sampling_lon(float): resolution for longitude
        n_fire (int): Number of fires
        **kwargs():

        
    Returns:
        the numpy arrays, holding the gridded slstr frp information and latitude and longitude data
    """
    
    n_lat = int( (np.float32(lat_max) - np.float32(lat_min)) / sampling_lat ) + 1 # Number of rows per latitude sampling
    n_lon = int( (np.float32(lon_max) - np.float32(lon_min)) / sampling_lon ) + 1 # Number of lines per longitude sampling

    
    slstr_frp_gridded = np.zeros( [n_lat, n_lon], dtype='float32' ) - 9999.

    lat_grid = np.zeros( [n_lat, n_lon], dtype='float32' ) - 9999.
    lon_grid = np.zeros( [n_lat, n_lon], dtype='float32' ) - 9999.
    
    if (n_fire >= 0):
    
    # Loop on i_lat: begins
        for i_lat in range(n_lat):
                    
        # Loop on i_lon: begins
            for i_lon in range(n_lon):
                        
                lat_grid[i_lat, i_lon] = lat_min + np.float32(i_lat) * sampling_lat + sampling_lat / 2.
                lon_grid[i_lat, i_lon] = lon_min + np.float32(i_lon) * sampling_lon + sampling_lon / 2.
                            
            # Gridded SLSTR FRP MWIR Night - All days
                if(parameter=='swir_nosaa'):
                    FLAG_FRP_SWIR_SAA_nc = kwargs.get('flag', None)
                    mask_grid = np.where( 
                        (lat_frp[:] >= lat_min + np.float32(i_lat) * sampling_lat)  & 
                        (lat_frp[:] < lat_min + np.float32(i_lat+1) * sampling_lat) & 
                        (lon_frp[:] >= lon_min + np.float32(i_lon) * sampling_lon)  & 
                        (lon_frp[:] < lon_min + np.float32(i_lon+1) * sampling_lon) &
                        (parameter_array[:] != -1.) & (FLAG_FRP_SWIR_SAA_nc[:] == 0), False, True)
                else:
                    mask_grid = np.where( 
                        (lat_frp[:] >= lat_min + np.float32(i_lat) * sampling_lat)  & 
                        (lat_frp[:] < lat_min + np.float32(i_lat+1) * sampling_lat) & 
                        (lon_frp[:] >= lon_min + np.float32(i_lon) * sampling_lon)  & 
                        (lon_frp[:] < lon_min + np.float32(i_lon+1) * sampling_lon) &
                        (parameter_array[:] != -1.),  False, True)
                            
                masked_slstr_frp_grid = np.ma.array(parameter_array[:], mask=mask_grid)
                            
                if len(masked_slstr_frp_grid.compressed()) != 0:
                    slstr_frp_gridded[i_lat, i_lon]  = np.sum(masked_slstr_frp_grid.compressed())
    return slstr_frp_gridded, lat_grid, lon_grid

<hr>

## <a id="visualization"></a>Data visualization functions

### <a id='visualize_pcolormesh'></a>`visualize_pcolormesh`

In [1]:
def visualize_pcolormesh(aod_ocean, aod_land, latitude, longitude, title, unit, vmin, vmax, color_scale, projection):
    """ 
    Visualizes Sentinel-3 SLSTR Aerosol Optical Depth data with the help of matplotlib's pcolormesh function.
    
    Parameters:
        aod_ocean (xarray DataArray): DataArray with AOD data over ocean
        aod_land (xarray DataArray): DataArray with AOD data over land
        latitude (xarray DataArray): latitude information retrieved from the data file
        longitude (xarray DataArray): longitude information retrieved from the data file
        title (str): Title of the plot
        unit (str): Unit of the data
        vmin (float): Minimum value for visualization
        vmax (float): Maximum value for visualization
        color_scale (str): Color scale
        projection (ccrs.projection): Projection of final plot

    """
    fig=plt.figure(figsize=(12, 12))

    ax=plt.axes(projection=projection)
    ax.coastlines(linewidth=1.5, linestyle='solid', color='k', zorder=10)

    gl = ax.gridlines(draw_labels=True, linestyle='--')
    gl.xlabels_top=False
    gl.ylabels_right=False
    gl.xformatter=LONGITUDE_FORMATTER
    gl.yformatter=LATITUDE_FORMATTER
    gl.xlabel_style={'size':12}
    gl.ylabel_style={'size':12}


    img1 = plt.pcolormesh(longitude, latitude, aod_ocean, transform=ccrs.PlateCarree(), vmin=vmin, vmax=vmax, cmap=color_scale)
    img2 = plt.pcolormesh(longitude, latitude, aod_land, transform=ccrs.PlateCarree(), vmin=vmax, vmax=vmax, cmap=color_scale)
    ax.set_title(title, fontsize=20, pad=20.0)

    cbar = fig.colorbar(img1, ax=ax, orientation='vertical', fraction=0.04, pad=0.05)
    cbar.set_label(unit, fontsize=16)
    cbar.ax.tick_params(labelsize=14)

    plt.show()

### <a id='vis_s3_frp'></a> `visualize_s3_frp`

In [1]:
def visualize_s3_frp(data, lat, lon, unit, longname, textstr_1, textstr_2, vmax):
    """ 
    Visualizes NRT Sentinel-3 SLSTR Fire Radiative Power data with the help of matplotlib's pcolormesh function.
    
    Parameters:
        data (numpy masked data array): dara array
        lat (numpy array): latitude information returned from the function `slstr_frp_gridding`
        lon (numpy array): longitude information returned from the function `slstr_frp_gridding`
        unit (str): Unit of the data
        longname (str): Long name attribute from the data or title of the plot
        textstr_1 (str): Text string that explains the number of hotspots
        textstr_2 (str): Text string that explains summary statistics of the data visualized.
        vmax (float): Maximum value for visualization.
    """
    fig=plt.figure(figsize=(20, 15))

    ax = plt.axes(projection=ccrs.PlateCarree())

    img = plt.pcolormesh(lon, lat, data, 
                        cmap=cm.autumn_r, transform=ccrs.PlateCarree(),
                        vmin=0,
                        vmax=vmax)

    ax.add_feature(cfeature.BORDERS, edgecolor='black', linewidth=1)
    ax.add_feature(cfeature.COASTLINE, edgecolor='black', linewidth=1)

    gl = ax.gridlines(draw_labels=True, linestyle='--')
    gl.xlabels_bottom=False
    gl.ylabels_right=False
    gl.xformatter=LONGITUDE_FORMATTER
    gl.yformatter=LATITUDE_FORMATTER
    gl.xlabel_style={'size':14}
    gl.ylabel_style={'size':14}

    cbar = fig.colorbar(img, ax=ax, orientation='horizontal', fraction=0.029, pad=0.025)
    cbar.set_label(unit, fontsize=16)
    cbar.ax.tick_params(labelsize=14)
    ax.set_title(longname, fontsize=20, pad=40.0) 

    props = dict(boxstyle='square', facecolor='white', alpha=0.5)

    # place a text box on the right side of the plot
    ax.text(1.1, 0.9, textstr_1, transform=ax.transAxes, fontsize=16,
        verticalalignment='top', bbox=props)

    props = dict(boxstyle='square', facecolor='white', alpha=0.5)

    # place a text box in upper left in axes coords
    ax.text(1.1, 0.85, textstr_2, transform=ax.transAxes, fontsize=16,
            verticalalignment='top', bbox=props)
    plt.show()


<hr>

<img src='./img/copernicus_logo.png' alt='Logo EU Copernicus' align='right' width='20%'><br><br><br>

<p style="text-align:right;">This project is licensed under the <a href="./LICENSE">MIT License</a> and is developed under a Copernicus contract.